# Identification of a patient cohort based on inclusion criteria

A patient cohort undergoing septic shock during admissions was created from the [MIMIC III database](https://mimic.physionet.org/mimictables/patients/), which contains information on patient stays in the Intensive Care Unit of the Beth Israel Deaconess Medical Center (Boston, MA). Inclusion of patients in the cohort is determined by a set of "inclusion criteria" that qualify the time and eligibility of patients relevant to the study.

The cohort building process is based on the 2015 paper: ["A targeted real-time early warning score (TREWScore) for septic shock" by Henry et al.](https://pubmed.ncbi.nlm.nih.gov/26246167/), with assistance in defining organ dysfunction using ICD9 codes from the 2001 paper: ["Epidemiology of severe sepsis in the United States: analysis of incidence, outcome, and associated costs of care" by Angus et al.](https://pubmed.ncbi.nlm.nih.gov/11445675/). Extraction of data for the septic shock cohort has already been done and converted to multiple csv files. The queries were done on the following MIMIC tables: 'ADMISSIONS', 'PATIENTS', CHARTEVENTS', 'LABEVENTS', 'D_LABITEMS'.

In [1]:
# Import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Set timezone to UTC
import os, time
os.environ['TZ'] = 'UTC'

import datetime
print(datetime.datetime.fromtimestamp(0).strftime('%Y-%m-%d %H:%M:%S'))

In [2]:
# Load the data
vitals_cohort_sirs = pd.read_csv('vitals_cohort_sirs.csv')
labs_cohort = pd.read_csv('labs_cohort.csv')

In [3]:
vitals_cohort_sirs.head()

,subject_id,hadm_id,icustay_id,charttime,valuenum,vital_id
0,3,145834,211552,2101-10-20 19:15:00,217.0,SysBP
1,3,145834,211552,2101-10-20 19:30:00,151.0,HeartRate
2,3,145834,211552,2101-10-20 19:30:00,102.0,SysBP
3,3,145834,211552,2101-10-20 19:45:00,135.0,HeartRate
4,3,145834,211552,2101-10-20 19:45:00,94.0,SysBP


In [4]:
vitals_cohort_sirs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20633223 entries, 0 to 20633222
Data columns (total 6 columns):
subject_id    int64
hadm_id       int64
icustay_id    int64
charttime     object
valuenum      float64
vital_id      object
dtypes: float64(1), int64(3), object(2)
memory usage: 944.5+ MB


In [5]:
labs_cohort.head()

,subject_id,hadm_id,icustay_id,charttime,lab_id,valuenum
0,3,145834,211552,2101-10-20 16:40:00,ANION GAP,17.0
1,3,145834,211552,2101-10-20 16:40:00,BANDS,2.0
2,3,145834,211552,2101-10-20 16:40:00,BICARBONATE,25.0
3,3,145834,211552,2101-10-20 16:40:00,BUN,53.0
4,3,145834,211552,2101-10-20 16:40:00,CHLORIDE,99.0


In [6]:
labs_cohort.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9947103 entries, 0 to 9947102
Data columns (total 6 columns):
subject_id    int64
hadm_id       int64
icustay_id    int64
charttime     object
lab_id        object
valuenum      float64
dtypes: float64(1), int64(3), object(2)
memory usage: 455.3+ MB


## Create Development Set for proof-of-concept

In [7]:
# Get the first 1,000 observations for each dataframe
vitals_cohort_sirs = vitals_cohort_sirs.head(1000)
labs_cohort = labs_cohort.head(1000)

## Systemic Inflammatory Response Syndrome (SIRS) criteria

The four SIRS criteria are as follows:
1. Temperature > 38 degC or < 36 degC
2. Heart Rate > 90
3. Respiratory Rate > 20 or PaCO$_{2}$< 32mmHg
4. WBC > 12,000/$mm^{3}$, < 4000/$mm^{3}$, or > 10% bands

According to the TREWScore paper, patient is considered to have sepsis if at least two of the four SIRS criteria are simultaneously met during an admission and there is a suspicion of infection.

In [8]:
# Calculate mean value for each vital_id and lab_id per patient and charttime
mean_vitals_ch = vitals_cohort_sirs.groupby(['subject_id', 'hadm_id', 'icustay_id', 'charttime', 'vital_id']).mean()
mean_vitals_ch.rename({'valuenum': 'mean_val'}, axis=1, inplace=True)

mean_labs_ch = labs_cohort.groupby(['subject_id', 'hadm_id', 'icustay_id', 'charttime', 'lab_id']).mean()
mean_labs_ch.rename({'valuenum': 'mean_val'}, axis=1, inplace=True)

In [9]:
# Get distinct mean values for each vital_id per patient and charttime
# Spread the dataframe by vital_id
mean_vitals_ch.drop_duplicates(subset=['mean_val'], inplace=True)
mean_vitals_ch = pd.pivot_table(mean_vitals_ch, values= 'mean_val',
                                index=['subject_id', 'hadm_id', 'icustay_id', 'charttime'],
                                columns=['vital_id'])
mean_vitals_ch.reset_index(inplace=True)

In [10]:
mean_vitals_ch.head()

vital_id,subject_id,hadm_id,icustay_id,charttime,HeartRate,RespRate,SysBP,TempC
0,3,145834,211552,2101-10-20 19:15:00,NaN,NaN,217.0,NaN
1,3,145834,211552,2101-10-20 19:30:00,151.0,NaN,102.0,NaN
2,3,145834,211552,2101-10-20 19:45:00,135.0,NaN,94.0,NaN
3,3,145834,211552,2101-10-20 20:00:00,143.0,24.0,79.0,NaN
4,3,145834,211552,2101-10-20 20:15:00,165.0,5.0,82.0,NaN


In [11]:
mean_vitals_ch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 8 columns):
subject_id    137 non-null int64
hadm_id       137 non-null int64
icustay_id    137 non-null int64
charttime     137 non-null object
HeartRate     46 non-null float64
RespRate      31 non-null float64
SysBP         70 non-null float64
TempC         45 non-null float64
dtypes: float64(4), int64(3), object(1)
memory usage: 8.6+ KB


In [12]:
# Get distinct mean values for each lab_id per patient and charttime
# Spread the dataframe by lab_id

mean_labs_ch.drop_duplicates(subset=['mean_val'], inplace=True)
mean_labs_ch = pd.pivot_table(mean_labs_ch, values= 'mean_val',
                              index=['subject_id', 'hadm_id', 'icustay_id', 'charttime'],
                              columns=['lab_id'])
mean_labs_ch.reset_index(inplace=True)

In [13]:
mean_labs_ch.head()

lab_id,subject_id,hadm_id,icustay_id,charttime,ALBUMIN,ANION GAP,BANDS,BICARBONATE,BILIRUBIN,BUN,...,HEMOGLOBIN,INR,LACTATE,PLATELET,POTASSIUM,PT,PTT,PaCO2,SODIUM,WBC
0,3,145834,211552,2101-10-20 16:40:00,NaN,17.0,2.0,25.0,NaN,53.0,...,10.0,1.3,NaN,282.0,5.4,13.5,30.7,NaN,136.0,12.7
1,3,145834,211552,2101-10-20 16:49:00,NaN,NaN,NaN,NaN,NaN,NaN,...,10.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,145834,211552,2101-10-20 19:12:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.3,NaN,4.4,NaN,NaN,40.0,138.0,NaN
3,3,145834,211552,2101-10-20 19:14:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.8,NaN,4.0,NaN,NaN,28.0,153.0,NaN
4,3,145834,211552,2101-10-20 19:26:00,NaN,23.0,NaN,13.0,NaN,41.0,...,7.8,1.7,NaN,190.0,NaN,15.7,58.3,NaN,143.0,11.3


In [14]:
mean_labs_ch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 24 columns):
subject_id     123 non-null int64
hadm_id        123 non-null int64
icustay_id     123 non-null int64
charttime      123 non-null object
ALBUMIN        5 non-null float64
ANION GAP      8 non-null float64
BANDS          1 non-null float64
BICARBONATE    6 non-null float64
BILIRUBIN      3 non-null float64
BUN            12 non-null float64
CHLORIDE       12 non-null float64
CREATININE     13 non-null float64
GLUCOSE        46 non-null float64
HEMATOCRIT     53 non-null float64
HEMOGLOBIN     31 non-null float64
INR            6 non-null float64
LACTATE        9 non-null float64
PLATELET       43 non-null float64
POTASSIUM      17 non-null float64
PT             18 non-null float64
PTT            49 non-null float64
PaCO2          13 non-null float64
SODIUM         14 non-null float64
WBC            27 non-null float64
dtypes: float64(20), int64(3), object(1)
memory usage: 23.1+ KB


### Complete patient timeline

In [15]:
# Full join on labs and vitals dataframes
labs_vitals = pd.merge(mean_vitals_ch, mean_labs_ch, on=['subject_id', 'hadm_id', 'icustay_id', 'charttime'],
                       how='outer')

In [16]:
labs_vitals.head()

,subject_id,hadm_id,icustay_id,charttime,HeartRate,RespRate,SysBP,TempC,ALBUMIN,ANION GAP,...,HEMOGLOBIN,INR,LACTATE,PLATELET,POTASSIUM,PT,PTT,PaCO2,SODIUM,WBC
0,3,145834,211552,2101-10-20 19:15:00,NaN,NaN,217.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,145834,211552,2101-10-20 19:30:00,151.0,NaN,102.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,145834,211552,2101-10-20 19:45:00,135.0,NaN,94.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,145834,211552,2101-10-20 20:00:00,143.0,24.0,79.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,145834,211552,2101-10-20 20:15:00,165.0,5.0,82.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Replace each missing value per icustay_id with the most recent prior present value (if any)
# Use last-observation-carried-forward (LOCF) approach
labs_vitals_2 = labs_vitals.groupby('icustay_id', as_index=False).fillna(method='ffill')

In [18]:
labs_vitals_2.head()

,subject_id,hadm_id,icustay_id,charttime,HeartRate,RespRate,SysBP,TempC,ALBUMIN,ANION GAP,...,HEMOGLOBIN,INR,LACTATE,PLATELET,POTASSIUM,PT,PTT,PaCO2,SODIUM,WBC
0,3,145834,211552,2101-10-20 19:15:00,NaN,NaN,217.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,145834,211552,2101-10-20 19:30:00,151.0,NaN,102.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,145834,211552,2101-10-20 19:45:00,135.0,NaN,94.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,145834,211552,2101-10-20 20:00:00,143.0,24.0,79.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,145834,211552,2101-10-20 20:15:00,165.0,5.0,82.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Create a short version of labs_vitals_2
labs_vitals_short = labs_vitals_2[['subject_id', 'hadm_id', 'icustay_id', 'charttime']]

In [20]:
# Count total NA's in dataframe before LOCF
labs_vitals.isna().sum()

subject_id       0
hadm_id          0
icustay_id       0
charttime        0
HeartRate      203
RespRate       218
SysBP          179
TempC          204
ALBUMIN        244
ANION GAP      241
BANDS          248
BICARBONATE    243
BILIRUBIN      246
BUN            237
CHLORIDE       237
CREATININE     236
GLUCOSE        203
HEMATOCRIT     196
HEMOGLOBIN     218
INR            243
LACTATE        240
PLATELET       206
POTASSIUM      232
PT             231
PTT            200
PaCO2          236
SODIUM         235
WBC            222
dtype: int64

In [21]:
# Count total NA's in dataframe after LOCF
labs_vitals_2.isna().sum()

subject_id       0
hadm_id          0
icustay_id       0
charttime        0
HeartRate       38
RespRate        28
SysBP           35
TempC           46
ALBUMIN        149
ANION GAP      129
BANDS          209
BICARBONATE    187
BILIRUBIN      172
BUN             71
CHLORIDE       101
CREATININE      97
GLUCOSE         43
HEMATOCRIT      41
HEMOGLOBIN      70
INR             64
LACTATE        203
PLATELET        63
POTASSIUM      160
PT              71
PTT             58
PaCO2          143
SODIUM         105
WBC             77
dtype: int64

### Compute SIRS criteria

In [22]:
# Construct boolean dataframe of all four criteria
sirs_tally_df = pd.DataFrame(
  {'sirs_1':((labs_vitals_2['TempC'] > 38) | (labs_vitals_2['TempC'] < 36)),
   'sirs_2':(labs_vitals_2['HeartRate'] > 90),
   'sirs_3':((labs_vitals_2['RespRate'] > 20) | (labs_vitals_2['PaCO2'] < 32)),
   'sirs_4':((labs_vitals_2['WBC'] > 12) | (labs_vitals_2['WBC'] < 4) |
             (labs_vitals_2['WBC'] > (0.1*labs_vitals_2['BANDS'])))
  }
)

In [23]:
sirs_tally_df.head()

,sirs_1,sirs_2,sirs_3,sirs_4
0,False,False,False,False
1,False,True,False,False
2,False,True,False,False
3,False,True,True,False
4,False,True,False,False


In [24]:
# Sum boolean values per observation to see if all four criteria or at least 2 of the 4 criteria are satisfied
# Add to labs_vitals_short
sirs = labs_vitals_short.copy()
sirs['all_4sirs'] = np.where(sirs_tally_df.select_dtypes(include=['bool']).sum(axis=1) == 4, True, False)
sirs['atleast_2sirs'] = np.where(sirs_tally_df.select_dtypes(include=['bool']).sum(axis=1) >= 2, True, False)

**Relevant infection codes are provided from the TREWScore paper**

In [ ]:
# Load the data
diagnoses = pd.read_csv('diagnoses.csv')

In [25]:
# Infection codes
infection3digit = ['001','002','003','004','005','008','009','010','011','012','013','014','015','016','017',
                   '018','020','021','022','023','024','025','026','027','030','031','032','033','034','035',
                   '036','037','038','039','040','041','090','091','092','093','094','095','096','097','098',
                   '100','101','102','103','104','110','111','112','114','115','116','117','118','320','322',
                   '324','325','420','421','451','461','462','463','464','465','481','482','485','486','494',
                   '510','513','540','541','542','566','567','590','597','601','614','615','616','681','682',
                   '683','686','730']

infection4digit = ['5695','5720','5721','5750','5990','7110','7907','9966','9985','9993']

infection5digit = ['49121','56201','56203','56211','56213', '56983']

infection_codes = infection3digit + infection4digit + infection5digit

In [26]:
# Filter rows based on ICD9 codes that start with the infection codes
infection = diagnoses[diagnoses['icd9_code'].str.startswith(tuple(infection_codes)).fillna(False)].reset_index()

In [27]:
infection.head()

,index,row_id,subject_id,hadm_id,seq_num,icd9_code,mimic_id
0,6,10,3,145834,7.0,6826,335873226
1,13,17,4,185777,5.0,7907,335873233
2,73,99,19,109235,2.0,5990,335873315
3,99,125,21,109451,14.0,1122,335873341
4,110,136,21,111970,7.0,6823,335873352


In [28]:
# Create short version of infection
infection_short = infection[['subject_id', 'hadm_id', 'icd9_code']]

In [29]:
infection_short.head()

,subject_id,hadm_id,icd9_code
0,3,145834,6826
1,4,185777,7907
2,19,109235,5990
3,21,109451,1122
4,21,111970,6823


## Identify admissions that mention the terms 'sepsis' or 'septic'

In [30]:
# Load the data
notes = pd.read_csv('notes_small_cohort_v2.csv')

In [31]:
# Filter rows based on note_text that contains 'sepsis' or 'septic'
sepsis_notes = notes[notes['note_text'].str.lower().str.contains(pat = 'sepsis|septic')].reset_index()

In [32]:
sepsis_notes.head()

,index,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,note_text
0,0,44005,3,145834,2101-10-31,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2101-10-20**] Discharg...
1,13,1260699,3,145834,2101-10-25,2101-10-25 06:05:00,2101-10-25 06:18:00,Nursing/other,Report,16302.0,NaN,MICU NPN 7P-7A\nNEURO: PATIENT ALERT AND ORIEN...
2,22,769247,3,145834,2101-10-26,2101-10-26 11:34:00,NaN,Radiology,VIDEO OROPHARYNGEAL SWALLOW,NaN,NaN,[**2101-10-26**] 11:34 AM\n VIDEO OROPHARYNGEA...
3,230,903363,21,111970,2135-01-31,2135-01-31 18:37:00,NaN,Radiology,CHEST PORT. LINE PLACEMENT,NaN,NaN,[**2135-1-31**] 6:37 PM\n CHEST PORT. LINE PLA...
4,231,903426,21,111970,2135-02-01,2135-02-01 09:16:00,NaN,Radiology,CT PELVIS W/CONTRAST,NaN,NaN,[**2135-2-1**] 9:16 AM\n CT PELVIS W/CONTRAST;...


In [33]:
sepsis_notes_short = sepsis_notes[['subject_id', 'hadm_id', 'charttime', 'note_text']]

In [34]:
sepsis_notes_short.head()

,subject_id,hadm_id,charttime,note_text
0,3,145834,NaN,Admission Date: [**2101-10-20**] Discharg...
1,3,145834,2101-10-25 06:05:00,MICU NPN 7P-7A\nNEURO: PATIENT ALERT AND ORIEN...
2,3,145834,2101-10-26 11:34:00,[**2101-10-26**] 11:34 AM\n VIDEO OROPHARYNGEA...
3,21,111970,2135-01-31 18:37:00,[**2135-1-31**] 6:37 PM\n CHEST PORT. LINE PLA...
4,21,111970,2135-02-01 09:16:00,[**2135-2-1**] 9:16 AM\n CT PELVIS W/CONTRAST;...


## Chart times and definition of sepsis

In [35]:
# Join sirs, infection, and notes dataframes and relabel the labs-vitals and notes charttimes
sepsis = sirs.merge(infection_short, on=['subject_id', 'hadm_id'], how='outer'
                   ).merge(sepsis_notes_short, on=['subject_id', 'hadm_id'], how='outer'
                          ).rename(columns={'charttime_x':'labvtl_charttime',
                                            'charttime_y':'notes_charttime'})

In [36]:
sepsis.head()

,subject_id,hadm_id,icustay_id,labvtl_charttime,all_4sirs,atleast_2sirs,icd9_code,notes_charttime,note_text
0,3,145834,211552.0,2101-10-20 19:15:00,False,False,6826,NaN,Admission Date: [**2101-10-20**] Discharg...
1,3,145834,211552.0,2101-10-20 19:15:00,False,False,6826,2101-10-25 06:05:00,MICU NPN 7P-7A\nNEURO: PATIENT ALERT AND ORIEN...
2,3,145834,211552.0,2101-10-20 19:15:00,False,False,6826,2101-10-26 11:34:00,[**2101-10-26**] 11:34 AM\n VIDEO OROPHARYNGEA...
3,3,145834,211552.0,2101-10-20 19:30:00,False,False,6826,NaN,Admission Date: [**2101-10-20**] Discharg...
4,3,145834,211552.0,2101-10-20 19:30:00,False,False,6826,2101-10-25 06:05:00,MICU NPN 7P-7A\nNEURO: PATIENT ALERT AND ORIEN...


In [37]:
# Replace each missing value per icustay_id with the most recent prior present value (if any)
# Use last-observation-carried-forward (LOCF) approach
sepsis_locf = sepsis.groupby('icustay_id', as_index=False).fillna(method='ffill')

In [40]:
# Sepsis definition is when at least 2 SIRS are present and
# icd9_code filtered for infection OR note_text filtered for 'sepsis', 'septic' contain values (no NA's)
sepsis_label = sepsis_locf.copy()
sepsis_label['sepsis_def'] = (sepsis_label['atleast_2sirs']==True) & (pd.notnull(sepsis_label['icd9_code']) |
                                                                      pd.notnull(sepsis_label['note_text']))

In [44]:
sepsis_label.head()

,subject_id,hadm_id,icustay_id,labvtl_charttime,all_4sirs,atleast_2sirs,icd9_code,notes_charttime,note_text,sepsis_def
0,3.0,145834.0,211552.0,2101-10-20 19:15:00,False,False,6826,NaN,Admission Date: [**2101-10-20**] Discharg...,False
1,3.0,145834.0,211552.0,2101-10-20 19:15:00,False,False,6826,2101-10-25 06:05:00,MICU NPN 7P-7A\nNEURO: PATIENT ALERT AND ORIEN...,False
2,3.0,145834.0,211552.0,2101-10-20 19:15:00,False,False,6826,2101-10-26 11:34:00,[**2101-10-26**] 11:34 AM\n VIDEO OROPHARYNGEA...,False
3,3.0,145834.0,211552.0,2101-10-20 19:30:00,False,False,6826,2101-10-26 11:34:00,Admission Date: [**2101-10-20**] Discharg...,False
4,3.0,145834.0,211552.0,2101-10-20 19:30:00,False,False,6826,2101-10-25 06:05:00,MICU NPN 7P-7A\nNEURO: PATIENT ALERT AND ORIEN...,False


## Chart times and admissions that meet the criteria for organ dysfunction and severe sepsis

**Relevant codes for sepsis-related organ dysfunction**

In [45]:
# ICD9 code prefixes for sepsis-related organ dysfunction (proxy)
code_prefixes = ['458','293','570','584', '7855','3483','3481', '2874','2875','2869','2866','5734']

In [54]:
# Add label column for severe sepsis
severe_sepsis = sepsis_label.copy()
severe_sepsis['severe_sep_def'] = np.where(
    (severe_sepsis['sepsis_def']==True) &
    (severe_sepsis['icd9_code'].str.startswith(tuple(code_prefixes))),
    True, False)

In [56]:
severe_sepsis.head()

,subject_id,hadm_id,icustay_id,labvtl_charttime,all_4sirs,atleast_2sirs,icd9_code,notes_charttime,note_text,sepsis_def,severe_sep_def
0,3.0,145834.0,211552.0,2101-10-20 19:15:00,False,False,6826,NaN,Admission Date: [**2101-10-20**] Discharg...,False,False
1,3.0,145834.0,211552.0,2101-10-20 19:15:00,False,False,6826,2101-10-25 06:05:00,MICU NPN 7P-7A\nNEURO: PATIENT ALERT AND ORIEN...,False,False
2,3.0,145834.0,211552.0,2101-10-20 19:15:00,False,False,6826,2101-10-26 11:34:00,[**2101-10-26**] 11:34 AM\n VIDEO OROPHARYNGEA...,False,False
3,3.0,145834.0,211552.0,2101-10-20 19:30:00,False,False,6826,2101-10-26 11:34:00,Admission Date: [**2101-10-20**] Discharg...,False,False
4,3.0,145834.0,211552.0,2101-10-20 19:30:00,False,False,6826,2101-10-25 06:05:00,MICU NPN 7P-7A\nNEURO: PATIENT ALERT AND ORIEN...,False,False


## Patient timeline for sepsis, severe sepsis, and septic shock

In [73]:
# Load the data
fluids_all = pd.read_csv('fluids_all.csv')

In [58]:
fluids_all.head()

,subject_id,hadm_id,icustay_id,charttime,amount_24h,current_amount,relative_amount,adequate_fluid
0,3,145834.0,211552.0,2101-10-21 00:00:00,10000.000000,10000.000000,103.305785,True
1,3,145834.0,211552.0,2101-10-21 02:00:00,10375.000000,375.000000,107.179752,True
2,3,145834.0,211552.0,2101-10-21 03:00:00,10625.000000,250.000000,109.762397,True
3,3,145834.0,211552.0,2101-10-21 04:00:00,10979.000000,354.000000,113.419421,True
4,3,145834.0,211552.0,2101-10-21 05:00:00,11364.004001,385.004001,117.396736,True


In [60]:
# Load the data
hypotension_label = pd.read_csv('hypotension_labels.csv')

In [61]:
hypotension_label.head()

,subject_id,hadm_id,icustay_id,charttime,hypotension
0,3,145834,211552,2101-10-20 16:40:00,False
1,3,145834,211552,2101-10-20 16:49:00,False
2,3,145834,211552,2101-10-20 19:12:00,False
3,3,145834,211552,2101-10-20 19:14:00,False
4,3,145834,211552,2101-10-20 19:15:00,False


In [76]:
# Select columns from fluids_all for joining by and to get adequate fluid label and charttime
# Rename to fluids_charttime
adq_fluids_label = fluids_all[['subject_id', 'hadm_id', 'icustay_id', 'adequate_fluid', 'charttime']]
adq_fluids_label.rename(columns={'charttime':'fluids_charttime'}, inplace=True)

# Select columns from fluids_all for joining by and to get hypotension label and charttime
# Rename to ht_charttime
label_hypot = hypotension_label[['subject_id', 'hadm_id', 'icustay_id', 'hypotension', 'charttime']]
label_hypot.rename(columns={'charttime':'ht_charttime'}, inplace=True)

septic_shock = severe_sepsis.merge(adq_fluids_label, on=['subject_id', 'hadm_id', 'icustay_id'], how='inner'
                                  ).merge(label_hypot, on=['subject_id', 'hadm_id', 'icustay_id'], how='inner')

In [78]:
# Replace each missing value per icustay_id with the most recent prior present value (if any)
septic_shock_locf = septic_shock.groupby('icustay_id', as_index=False).fillna(method='ffill')

In [80]:
# Add septic shock definition label to table
septic_shock_label = septic_shock_locf.copy()
septic_shock_label['septic_shock_def'] = np.where(
    (septic_shock_label['severe_sep_def']==True) &
    (septic_shock_label['adequate_fluid']==True) &
    (septic_shock_label['hypotension']==True),
    True, False)

In [82]:
septic_shock_label.head()

,subject_id,hadm_id,icustay_id,labvtl_charttime,all_4sirs,atleast_2sirs,icd9_code,notes_charttime,note_text,sepsis_def,severe_sep_def,adequate_fluid,fluids_charttime,hypotension,ht_charttime,septic_shock_def
0,3.0,145834.0,211552.0,2101-10-20 19:15:00,False,False,6826,NaN,Admission Date: [**2101-10-20**] Discharg...,False,False,True,2101-10-21 00:00:00,False,2101-10-20 16:40:00,False
1,3.0,145834.0,211552.0,2101-10-20 19:15:00,False,False,6826,NaN,Admission Date: [**2101-10-20**] Discharg...,False,False,True,2101-10-21 00:00:00,False,2101-10-20 16:49:00,False
2,3.0,145834.0,211552.0,2101-10-20 19:15:00,False,False,6826,NaN,Admission Date: [**2101-10-20**] Discharg...,False,False,True,2101-10-21 00:00:00,False,2101-10-20 19:12:00,False
3,3.0,145834.0,211552.0,2101-10-20 19:15:00,False,False,6826,NaN,Admission Date: [**2101-10-20**] Discharg...,False,False,True,2101-10-21 00:00:00,False,2101-10-20 19:14:00,False
4,3.0,145834.0,211552.0,2101-10-20 19:15:00,False,False,6826,NaN,Admission Date: [**2101-10-20**] Discharg...,False,False,True,2101-10-21 00:00:00,False,2101-10-20 19:15:00,False
